# Query Police API by specific point and list of months
Search the [UK Police Data API for street-level crimes](https://data.police.uk/docs/method/crime-street/) by point (lat/lon) and list of months (YYYY-MM) for all or specific crime types. Returns a dataframe.

## Parameters:
Geocode given address/landmark.

Specify months and crime category to search for ('all-crime' returns all types for the given month).

Specify path for output file.

In [2]:
import pandas as pd
import requests
from geopy.geocoders import Nominatim
import json

In [3]:
# Geocode address/landmark with Nominatim (latitude, longitude)
input_location = 'Imperial College' # Where: Address or landmark (for geocoding)
locator = Nominatim(user_agent='PoliceData') #Specify geocoding service. The user_agent name is arbitrary
geo_location = locator.geocode(input_location)

In [4]:
# Police Data API: Months and crime category to search for:
months = ['2021-08']
# Specify crime type to search for
crime_type = 'all-crime'

## Get data from Police API, make dataframe
Query the given location, once for each month listed. Returns dataframe.

In [6]:
# Get the crime data for 1-mile radius around given lat/lon point
def getCrimePoint(location, month, crime):
    # URL and parameters to pass to Police API:
    url = ('https://data.police.uk/api/crimes-street')
    lat = location.latitude
    lng = location.longitude
    query = {'lat': lat, 'lng':lng, 'date':month}
    # Response-object from API request:
    r = requests.get(f"{url}/{crime}", params = query)
    return(r)

In [7]:
# Loop through the API response by date, using the getCrimePoly function. 
# Stack up responses as a list of lists of dicts.
crime_list = []
for month in months:
    crime = getCrimePoint(geo_location, month, crime_type)
    crime = crime.json() #Encode API response as JSON
    crime_list.append(crime) #Append response to list

In [8]:
# Flatten crime_list into a single list of dicts
# from https://stackoverflow.com/a/952952/6023102
flat_list = [item for sublist in crime_list for item in sublist]

In [9]:
# Make dataframe
df = pd.json_normalize(flat_list)

In [10]:
# Number of incidents in dataframe
len(df)

1199

In [11]:
df.keys()

Index(['category', 'location_type', 'context', 'outcome_status',
       'persistent_id', 'id', 'location_subtype', 'month', 'location.latitude',
       'location.street.id', 'location.street.name', 'location.longitude',
       'outcome_status.category', 'outcome_status.date'],
      dtype='object')

In [19]:
crime_types = df.groupby('category')['id'].agg('count').reset_index()
crime_types.sort_values(by='id', ascending=False, inplace=True)

In [20]:
crime_types

,category,id
0,anti-social-behaviour,249
12,violent-crime,211
6,other-theft,180
11,vehicle-crime,139
9,shoplifting,87
7,public-order,67
2,burglary,64
10,theft-from-the-person,58
1,bicycle-theft,39
3,criminal-damage-arson,34


## Save df to pickle
Speedy binary file format for loading into a separate notebook for analysis.

In [5]:
# Output file (if needed), pickle format:
path = r'data\police_points_month.pkl'

In [ ]:
# Pickle for use in different notebook for analysis
df.to_pickle(path)